<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Click-That-Recommendation-Button" data-toc-modified-id="Click-That-Recommendation-Button-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Click That Recommendation Button</a></span><ul class="toc-item"><li><span><a href="#Business-Understanding-and-Business-Problem" data-toc-modified-id="Business-Understanding-and-Business-Problem-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Business Understanding and Business Problem</a></span><ul class="toc-item"><li><span><a href="#Cost-of-Errors" data-toc-modified-id="Cost-of-Errors-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Cost of Errors</a></span></li></ul></li><li><span><a href="#Data-Understanding" data-toc-modified-id="Data-Understanding-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Understanding</a></span><ul class="toc-item"><li><span><a href="#Initial-Data" data-toc-modified-id="Initial-Data-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Initial Data</a></span></li></ul></li><li><span><a href="#Exploratory-Data-Analysis" data-toc-modified-id="Exploratory-Data-Analysis-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Exploratory Data Analysis</a></span><ul class="toc-item"><li><span><a href="#Exploring-links_df" data-toc-modified-id="Exploring-links_df-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Exploring <code>links_df</code></a></span></li><li><span><a href="#Exploring-movies_df" data-toc-modified-id="Exploring-movies_df-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Exploring <code>movies_df</code></a></span></li><li><span><a href="#Exploring-ratings_df" data-toc-modified-id="Exploring-ratings_df-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Exploring <code>ratings_df</code></a></span></li><li><span><a href="#Exploring-tags_df" data-toc-modified-id="Exploring-tags_df-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Exploring <code>tags_df</code></a></span></li><li><span><a href="#Creating-a-New-DataFrame-with-Relevant-Data" data-toc-modified-id="Creating-a-New-DataFrame-with-Relevant-Data-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Creating a New DataFrame with Relevant Data</a></span><ul class="toc-item"><li><span><a href="#Additional-EDA-that-likely-can-be-dropped-for-final-notebook" data-toc-modified-id="Additional-EDA-that-likely-can-be-dropped-for-final-notebook-1.3.5.1"><span class="toc-item-num">1.3.5.1&nbsp;&nbsp;</span><strong>Additional EDA that likely can be dropped for final notebook</strong></a></span></li></ul></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#First-Baseline-Model" data-toc-modified-id="First-Baseline-Model-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>First Baseline Model</a></span></li><li><span><a href="#Model-KNN-with-Z-Score" data-toc-modified-id="Model-KNN-with-Z-Score-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Model KNN with Z-Score</a></span></li><li><span><a href="#Model-with-SVD" data-toc-modified-id="Model-with-SVD-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Model with SVD</a></span></li><li><span><a href="#Model-with-SVD++" data-toc-modified-id="Model-with-SVD++-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Model with SVD++</a></span></li><li><span><a href="#All-Models?-(maybe-we-can-condense-the-modeling-section-so-its-not-a-bunch-of-the-same-code-again-and-again)" data-toc-modified-id="All-Models?-(maybe-we-can-condense-the-modeling-section-so-its-not-a-bunch-of-the-same-code-again-and-again)-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>All Models? (maybe we can condense the modeling section so its not a bunch of the same code again and again)</a></span><ul class="toc-item"><li><span><a href="#Gridsearch-best-model" data-toc-modified-id="Gridsearch-best-model-1.9.1"><span class="toc-item-num">1.9.1&nbsp;&nbsp;</span>Gridsearch best model</a></span></li></ul></li></ul></li><li><span><a href="#Creating-a-New-User" data-toc-modified-id="Creating-a-New-User-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a New User</a></span><ul class="toc-item"><li><span><a href="#Go-back-tomorrow-and-check-output-for-all-functions-to-see-why-it-isnt-saving-to-the-dataframe" data-toc-modified-id="Go-back-tomorrow-and-check-output-for-all-functions-to-see-why-it-isnt-saving-to-the-dataframe-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Go back tomorrow and check output for all functions to see why it isnt saving to the dataframe</a></span></li><li><span><a href="#making-predictions-with-surprise" data-toc-modified-id="making-predictions-with-surprise-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>making predictions with surprise</a></span></li><li><span><a href="#Final-Model" data-toc-modified-id="Final-Model-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Final Model</a></span></li><li><span><a href="#Results/Conclusions" data-toc-modified-id="Results/Conclusions-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Results/Conclusions</a></span><ul class="toc-item"><li><span><a href="#Further-Exploration/Questions" data-toc-modified-id="Further-Exploration/Questions-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Further Exploration/Questions</a></span></li></ul></li></ul></li></ul></div>

insert header

# Click That Recommendation Button

## Business Understanding and Business Problem

Our team has been tasked with implementation of a recommendation system for the movie rental company, RedBox. We have three main goals:

- Create a recommendation system model that allows users to input movie ratings and provides movie suggestions
- Be able to store user ratings for future recommendation requests
- Be able to provide specific genre recommendations based on a users request

### Cost of Errors

Cost of Errors is somewhat ephemeral for this project. The main cost of errors would be user trust and loyalty. 

While the goal is to build user loyalty and create a seamless user experience from recommendation to rental, if users don't find value in the recommendations provided than that loyalty is lost in an instant.

## Data Understanding

In [51]:
#imports grabbed from lecture
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import knns, SVD, BaselineOnly, SVDpp
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy


import warnings
warnings.filterwarnings('ignore')

### Initial Data

Our data was provided by MovieLens data set and includes over 100,000 ratings made by 600 users across almost 10,000 movies. The data is broken into four different .CSV files and further analysis of these files is discussed below.

In [2]:
#the four csv files provided
links_df = pd.read_csv('data/links.csv')
movies_df = pd.read_csv('data/movies.csv')
ratings_df = pd.read_csv('data/ratings.csv')
tags_df = pd.read_csv('data/tags.csv')

In [3]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


## Exploratory Data Analysis

### Exploring `links_df`

In [8]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


This file is a key to merge movie identifiers with IMDB Database and The Movie DataBase. We will be focusing on the MovieLens database for this analysis and will not need this file.

### Exploring `movies_df`

In [9]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
movies_df.shape

(9742, 3)

In [11]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [12]:
movies_df['title'].nunique()

9737

In [13]:
movies_df['title'].value_counts().head(10)

War of the Worlds (2005)                             2
Eros (2004)                                          2
Emma (1996)                                          2
Confessions of a Dangerous Mind (2002)               2
Saturn 3 (1980)                                      2
Great Beauty, The (Grande Bellezza, La) (2013)       1
'night Mother (1986)                                 1
Libertine, The (2004)                                1
St. Elmo's Fire (1985)                               1
Revenge of the Nerds II: Nerds in Paradise (1987)    1
Name: title, dtype: int64

There are 9737 unique movie titles. Repeats for 5 movies:
- Emma (1996)                                   2
- Saturn 3 (1980)                               2
- Eros (2004)                                   2
- Confessions of a Dangerous Mind (2002)        2
- War of the Worlds (2005)                      2

Cleaning the text in the DataFrame.

In [14]:
movies_df['split_genres'] = movies_df['genres'].iloc[0:].apply(lambda x: x.lower().split('|'))
movies_df['genres'] = movies_df['genres'].iloc[0:].apply(lambda x: x.lower())

This file will be very helpful for our analysis. It provides us with the title of movie in relation to its unique identifier and lets us know the genre categories that the movie would fall under.

### Exploring `ratings_df`

In [15]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [16]:
ratings_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [17]:
# The 10 movies with the most reviews
ratings_df.groupby('movieId').count().sort_values('userId', ascending=False).head(10)

,userId,rating,timestamp
movieId,,,
356,329,329,329
318,317,317,317
296,307,307,307
593,279,279,279
2571,278,278,278
260,251,251,251
480,238,238,238
110,237,237,237
589,224,224,224


In [18]:
# the 10 movies with the highest ratings
ratings_df.groupby('movieId').mean().sort_values('rating', ascending=False).head(10)

,userId,rating,timestamp
movieId,,,
88448,483.0,5.0,1.315438e+09
100556,462.0,5.0,1.456151e+09
143031,89.0,5.0,1.520409e+09
143511,105.0,5.0,1.526207e+09
143559,89.0,5.0,1.520410e+09
6201,474.0,5.0,1.100120e+09
102217,63.0,5.0,1.443200e+09
102084,380.0,5.0,1.493422e+09
6192,182.0,5.0,1.063275e+09


In [19]:
# the 10 users with the most reviews
ratings_df.groupby('userId').count().sort_values('rating', ascending=False).head(10)

,movieId,rating,timestamp
userId,,,
414,2698,2698,2698
599,2478,2478,2478
474,2108,2108,2108
448,1864,1864,1864
274,1346,1346,1346
610,1302,1302,1302
68,1260,1260,1260
380,1218,1218,1218
606,1115,1115,1115


This will be the primary datafile for this analysis. It includes relevant information including `userId`, `rating`, and `movieId`. `timestamp` is not relevant for this analysis and will be dropped.

In [20]:
ratings_df.drop('timestamp', axis=1, inplace=True)

### Exploring `tags_df`

In [21]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [22]:
tags_df.isna().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [23]:
# users with the most comments on movies
tags_df.groupby('userId').count().sort_values('tag', ascending=False).head(10)

,movieId,tag,timestamp
userId,,,
474,1507,1507,1507
567,432,432,432
62,370,370,370
599,323,323,323
477,280,280,280
424,273,273,273
537,100,100,100
125,48,48,48
357,45,45,45


This will be helpful for analysis as it provides keyword insights to each of the films. `timestamp` will not be helpful for this analysis and will be dropped later.

### Creating a New DataFrame with Relevant Data

To better understand and work with the separate files, they have been merged to one, larger working file. 

In [24]:
# Merging the ratings data with movie titles and genres
merged_df = ratings_df.merge(movies_df, how='outer', on='movieId')

In [25]:
merged_df.head()

,userId,movieId,rating,title,genres,split_genres
0,1.0,1,4.0,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"
1,5.0,1,4.0,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"
2,7.0,1,4.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"
3,15.0,1,2.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"
4,17.0,1,4.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"


In [26]:
# Checking for nulls after merging
merged_df.isna().sum()

userId          18
movieId          0
rating          18
title            0
genres           0
split_genres     0
dtype: int64

In [27]:
# 18 movies do not have any reviews 
merged_df.tail(20)

,userId,movieId,rating,title,genres,split_genres
100834,610.0,163937,3.5,Blair Witch (2016),horror|thriller,"[horror, thriller]"
100835,610.0,163981,3.5,31 (2016),horror,[horror]
100836,NaN,1076,NaN,"Innocents, The (1961)",drama|horror|thriller,"[drama, horror, thriller]"
100837,NaN,2939,NaN,Niagara (1953),drama|thriller,"[drama, thriller]"
100838,NaN,3338,NaN,For All Mankind (1989),documentary,[documentary]
100839,NaN,3456,NaN,"Color of Paradise, The (Rang-e khoda) (1999)",drama,[drama]
100840,NaN,4194,NaN,I Know Where I'm Going! (1945),drama|romance|war,"[drama, romance, war]"
100841,NaN,5721,NaN,"Chosen, The (1981)",drama,[drama]
100842,NaN,6668,NaN,"Road Home, The (Wo de fu qin mu qin) (1999)",drama|romance,"[drama, romance]"
100843,NaN,6849,NaN,Scrooge (1970),drama|fantasy|musical,"[drama, fantasy, musical]"


In [28]:
# Dropping nulls in data
merged_df.dropna(inplace=True)

In [29]:
merged_df.isna().sum()

userId          0
movieId         0
rating          0
title           0
genres          0
split_genres    0
dtype: int64

In [30]:
merged_df.shape

(100836, 6)

In [31]:
merged_df['movieId'].nunique()

9724

We have a cleaned dataframe with 100836 reviews for 9724 movies.

#### **Additional EDA that likely can be dropped for final notebook**

In [32]:
# creating a sparse matrix pivot table that has all users as the index and all movies as the columns
pivot_test = merged_df.pivot_table(index='userId', columns='title', values='rating')

In [33]:
pivot_test.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Modeling

We will be using the Surprise library for this analysis. This library requires that data inputs be limited to three columns of information:

- User
- Movie
- Rating

In [34]:
# Adjusting the dataframe for modeling purposes
model_df = merged_df.drop(['title', 'genres', 'split_genres'], axis=1)

In [35]:
# Contains all movie ratings and the unique identifiers for user and movie
model_df.head()

,userId,movieId,rating
0,1.0,1,4.0
1,5.0,1,4.0
2,7.0,1,4.5
3,15.0,1,2.5
4,17.0,1,4.5


In [36]:
# Converting model_df to surprise accessible data
reader = Reader()
data = Dataset.load_from_df(model_df,reader)

In [37]:
trainset, testset = train_test_split(data, test_size=.2, random_state=42)

In [38]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items)

Number of users:  610 

Number of items:  8982


## First Baseline Model

Setting up a baseline model using Surprise's `BaseLineOnly` algorithm. 

In [39]:
baseline = BaselineOnly()
baseline.fit(trainset)

Estimating biases using als...


In [40]:
base_predictions = baseline.test(testset)

In [41]:
accuracy.rmse(base_predictions)

RMSE: 0.8734


0.8734341181528673

In [42]:
accuracy.mae(base_predictions)

MAE:  0.6722


0.6722070558104862

In [62]:
cv = cross_validate(baseline, data, n_jobs=3)

In [93]:
results_df = pd.DataFrame()
results_dict = {'model': baseline}

for item in cv.items():
#     results_dict = {}
    results_dict[f"mean_{item[0]}"] = np.mean(item[-1])
    print(item[0])
    print(np.mean(item[-1]))

results_df.append(results_dict, ignore_index=True, sort=False)

test_rmse
0.8730027359247566
test_mae
0.6731495254534968
fit_time
0.033376455307006836
test_time
0.05573716163635254


,mean_fit_time,mean_test_mae,mean_test_rmse,mean_test_time,model
0,0.033376,0.67315,0.873003,0.055737,<surprise.prediction_algorithms.baseline_only....


## Model KNN with Z-Score

In [43]:
knn_z = knns.KNNWithZScore(random_state=42)
knn_z.fit(trainset=trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [44]:
knn_z_predictions = knn_z.test(testset)

In [45]:
accuracy.rmse(knn_z_predictions)

RMSE: 0.8963


0.8962709994505563

In [46]:
accuracy.mae(knn_z_predictions)

MAE:  0.6769


0.6769105071826177

## Model with SVD

In [47]:
svd = SVD(random_state=42)
svd.fit(trainset)

In [48]:
svd_predictions = svd.test(testset)

In [49]:
accuracy.rmse(svd_predictions)

RMSE: 0.8757


0.8757353176838333

In [50]:
accuracy.mae(svd_predictions)

MAE:  0.6715


0.6714792053984324

## Model with SVD++

In [52]:
svdpp = SVDpp(random_state=42)
svdpp.fit(trainset)

In [53]:
svdpp_predictions = svdpp.test(testset)

In [54]:
accuracy.rmse(svdpp_predictions)

RMSE: 0.8616


0.8616314766744503

In [55]:
accuracy.mae(svdpp_predictions)

MAE:  0.6585


0.6584658027784405

## All Models? (maybe we can condense the modeling section so its not a bunch of the same code again and again)

In [59]:
from surprise.prediction_algorithms import *
from surprise.model_selection import GridSearchCV, cross_validate

In [112]:
models_list = [NormalPredictor(), BaselineOnly(), KNNBaseline(random_state=42),
               KNNBasic(random_state=42), KNNWithMeans(random_state=42), 
               KNNWithZScore(random_state=42), SVD(random_state=42), 
               SVDpp(random_state=42), NMF(random_state=42), SlopeOne(), 
               CoClustering(random_state=42)]

In [113]:
results_df = pd.DataFrame()

for model in models_list:
    model_dict = {}
    temp_model = cross_validate(model, data, cv=3, n_jobs=3, verbose=False)
    model_dict['model'] = model
    for item in temp_model.items():
        model_dict[f"mean_{item[0]}"] = np.mean(item[-1])
    results_df = results_df.append(model_dict, ignore_index=True, sort=False)

In [114]:
results_df.sort_values('mean_test_mae')

,mean_fit_time,mean_test_mae,mean_test_rmse,mean_test_time,model
7,170.620822,0.667532,0.869729,4.667422,<surprise.prediction_algorithms.matrix_factori...
2,0.072381,0.674500,0.882325,1.273059,<surprise.prediction_algorithms.knns.KNNBaseli...
1,0.023875,0.675894,0.876393,0.090204,<surprise.prediction_algorithms.baseline_only....
6,1.805532,0.677758,0.880992,0.125624,<surprise.prediction_algorithms.matrix_factori...
5,0.069958,0.684488,0.902209,1.114337,<surprise.prediction_algorithms.knns.KNNWithZS...
4,0.054664,0.691516,0.905344,1.021725,<surprise.prediction_algorithms.knns.KNNWithMe...
9,1.782339,0.695004,0.907771,3.932988,<surprise.prediction_algorithms.slope_one.Slop...
8,1.966401,0.716263,0.933924,0.106362,<surprise.prediction_algorithms.matrix_factori...
3,0.048387,0.733830,0.957030,0.916583,<surprise.prediction_algorithms.knns.KNNBasic ...
10,0.837916,0.736870,0.952521,0.095379,<surprise.prediction_algorithms.co_clustering....


### Gridsearch best model

In [ ]:
# # Commenting out cause this is a work in progress

# sim_cos = {'name':'msd', 'user_based':False}
# params = {'n_factors': []}



# model = GridSearchCV(SVDpp()

# Creating a New User

In [48]:
merged_df.head()

,userId,movieId,rating,title,genres,split_genres
0,1.0,1,4.0,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"
1,5.0,1,4.0,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"
2,7.0,1,4.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"
3,15.0,1,2.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"
4,17.0,1,4.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]"


In [49]:
new_user_df = merged_df.copy()
new_user_df['username'] = np.nan
new_user_df['password'] = np.nan

In [50]:
new_user_df.tail()

,userId,movieId,rating,title,genres,split_genres,username,password
100831,610.0,160341,2.5,Bloodmoon (1997),action|thriller,"[action, thriller]",NaN,NaN
100832,610.0,160527,4.5,Sympathy for the Underdog (1971),action|crime|drama,"[action, crime, drama]",NaN,NaN
100833,610.0,160836,3.0,Hazard (2005),action|drama|thriller,"[action, drama, thriller]",NaN,NaN
100834,610.0,163937,3.5,Blair Witch (2016),horror|thriller,"[horror, thriller]",NaN,NaN
100835,610.0,163981,3.5,31 (2016),horror,[horror],NaN,NaN


In [51]:
test_user = {'userId':611, 
             'movieId':1, 
             'rating':5, 
             'title': 'Toy Story (1995)', 
             'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 
             'username': 'test_user', 
             'password': 'test_user'}

In [52]:
new_user_df = new_user_df.append(test_user, ignore_index=True)
new_user_df.tail()

,userId,movieId,rating,title,genres,split_genres,username,password
100832,610.0,160527,4.5,Sympathy for the Underdog (1971),action|crime|drama,"[action, crime, drama]",NaN,NaN
100833,610.0,160836,3.0,Hazard (2005),action|drama|thriller,"[action, drama, thriller]",NaN,NaN
100834,610.0,163937,3.5,Blair Witch (2016),horror|thriller,"[horror, thriller]",NaN,NaN
100835,610.0,163981,3.5,31 (2016),horror,[horror],NaN,NaN
100836,611.0,1,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,test_user,test_user


In [53]:
def new_user():
    # creating a while loop so that the program runs until everything is satisfied
    new_user_cleared = False
    while new_user_cleared is False:
        # checking if new user or not
        answer = input("(Type 'quit' at any time to exit.) \n\nAre you a new user? (yes/no): ")
        new_user_cleared = False
        # this section is for new users
        if answer == 'yes':
            ok_to_move_on = False
            while ok_to_move_on is False:
                # creating a new userId based on the last one available
                new_user_id = (new_user_df['userId'].max() + 1)
                # asking for them to create a username
                username = input('Please enter a unique username: ')
                # letting them quit the program if they want to stop
                if username == 'quit':
                    print('Thank you. Closing the program.')
                    user_id = np.NaN
                    username = np.NaN
                    password = np.NaN
                    new_user_cleared = True
                    ok_to_move_on = True
                    break
                else:
                    # Checking the list of usernames to see if its available
                    user_check = len(new_user_df[new_user_df['username'] == username]) <1
                    if user_check:
                        # having them make a new password
                        password = input('Please enter a password: ')
                        # letting them quit the program if they want to stop
                        if password == 'quit':
                            print('Thank you. Closing the program.')
                            user_id = np.NaN
                            username = np.NaN
                            password = np.NaN
                            new_user_cleared = True
                            ok_to_move_on = True
                            break
                        else:
                            # creating a new account with user input username and password
                            ok_to_move_on = True
                            new_user_cleared = True
                            print('\nWelcome! Your new account is created...\n')
                            user_id = new_user_id
                    else:
                        # returns them to make a new username if the name isnt available
                        print('Sorry, that name is taken.')
        # this section is for existing users    
        elif answer == 'no':
            ok_to_move_on = False
            while ok_to_move_on is False:
                username = input('Please enter your username: ')
                # letting them quit the program if they want to stop
                if username == 'quit':
                    print('Thank you. Closing the program.')
                    user_id = np.NaN
                    username = np.NaN
                    password = np.NaN
                    new_user_cleared = True
                    ok_to_move_on = True
                    break
                else:
                    # checking to make sure username is in the list of active users
                    user_check = new_user_df[new_user_df['username'] == username]['username'] == username
                    if len(user_check.values !=0):
                        password = input('Please enter your password: ')
                        # letting them quit the program if they want to stop
                        if password == 'quit':
                            print('Thank you. Closing the program.')
                            user_id = np.NaN
                            username = np.NaN
                            password = np.NaN
                            new_user_cleared = True
                            ok_to_move_on = True
                            break
                        else:
                            # checking password match with username
                            password_check = new_user_df[(new_user_df['username'] == username) & (new_user_df['password'] == password)]['password'] == password
                            if len(password_check.values) !=0:
                                user_id = new_user_df[(new_user_df['username'] == username) & (new_user_df['password'] == password)]['userId']
                                user_id = user_id.values[0]
                                print('...\n...\nLog-in Successful.')
                                ok_to_move_on = True
                                new_user_cleared = True
                            else:
                                # letting them know that there is no username password match
                                print('Sorry, that username and password is not in the system.')
                    else:
                        # letting them know that there is no username with that name in the system
                        print('Sorry, that username is not in the system.')
        # letting the user quit if they want to stop the program
        elif answer == 'quit':
            print('Thank you. Closing the program.')
            user_id = np.NaN
            username = np.NaN
            password = np.NaN
            break
            
        elif new_user_cleared == True:
            break
        else:
            # message if a user tries to enter anything but yes or no
            print('Invalid entry. Please enter (yes/no).\n')
          
    if user_id:
        
        return user_id, username, password

In [54]:
# u_id, u_name, p_word = new_user()

In [55]:
# creating a dataframe of the top 100 movies by review count
top_500 = new_user_df.groupby('movieId').count().sort_values('userId', ascending=False).head(500)
top_500 = top_500.reset_index()[['movieId']]

In [56]:
# adding in the movie title and genre
top_500 = top_500.merge(movies_df, on='movieId', how='left')
top_500.head()

,movieId,title,genres,split_genres
0,356,Forrest Gump (1994),comedy|drama|romance|war,"[comedy, drama, romance, war]"
1,318,"Shawshank Redemption, The (1994)",crime|drama,"[crime, drama]"
2,296,Pulp Fiction (1994),comedy|crime|drama|thriller,"[comedy, crime, drama, thriller]"
3,593,"Silence of the Lambs, The (1991)",crime|horror|thriller,"[crime, horror, thriller]"
4,2571,"Matrix, The (1999)",action|sci-fi|thriller,"[action, sci-fi, thriller]"


In [57]:
#  Newest one to coordinate better with other function


def movie_recs(user_id, username, password, main_df = new_user_df):
    # Setting the base rating at zero
    rating = 0
    # creating a new DF to hold the temp ratings
    current_session_df = pd.DataFrame()
    # whether or not its ok to quit the recommend prompt loop
    ok_to_quit = False

    # the loop to make recommendations
    while ok_to_quit is False:
        # prints message if they have less than 5 ratings
        if len(main_df[main_df['userId'] == user_id]) < 5:
            print("You need at least 5 reviews before we can make recommendations.")
        else:
            print('You are now ready to make additional recommendations.')
        # setting a loop to keep sampling if the sample is already in the main df or in the temp df
        ok_to_move_on = False
        while ok_to_move_on is False:
            # pulling sample from most rated moves
            movie_review = top_500.sample()
            movie_check = movie_review['movieId'].values
            # checking length of df to see if its been filled yet
            # ensuring movie has not already been rated
            if len(current_session_df) != 0:
                if movie_check not in main_df[main_df['userId'] == 
                                                  user_id]['movieId'].values and\
                movie_check not in current_session_df['movieId'].values:
                    ok_to_move_on = True
                # resample if already rated
                else:
                    continue
            else:
                # ensuring movie has not already been rated
                if movie_check not in main_df[main_df['userId'] == 
                                                  user_id]['movieId'].values:
                    ok_to_move_on = True
                else:
                    # resample if already rated
                    continue
        # creating a loop to ensure that user provides a valid rating
        while rating not in range(1,6):
            # creating a dictionary to store rating info that will be appended to df
            rating_dict = {'userId':user_id, 
                       'movieId': np.nan, 
                       'rating': np.nan, 
                       'title': np.nan, 
                       'genres': np.nan, 
                       'username': username, 
                       'password': password}
            print(f"\n(Type 'quit' to stop rating movies.) \n")
            # telling user how to rate
            print(f'Please rate the movie 1-5. If you have not seen the movie, type "n" to skip: \n')
            # giving user movie info
            print(f"{movie_review['title'].values}")
            rating = input('Rating: \n')
            # letting the user quit if they dont want to rate
            if rating == 'quit':
                print('\n\nThank you for ranking movies.')
                # checking to see if the user made any ratings during this session
                if len(current_session_df) != 0:
                    print('Here are the results of your ranking session:')
                    # showing the user the ratings they made
                    print(current_session_df[['title', 'rating']])
                    # checking if user wants to save ratings
                    submit_ratings = input('\nWould you like to save these ratings to your profile? (yes/quit): ')
                    if submit_ratings == 'yes':
                        # saving ratings to main database and closing program
                        temp_df = main_df.append(current_session_df, ignore_index=True)
                        ok_to_quit = True
                        ok_to_move_on = True
                        to_return = True
                        print('\nThank you. Your results have been saved to the main system!')
                        break
                    elif submit_ratings == 'quit':
                        # not saving ratings to main database and closing program
                        rating = 10
                        ok_to_move_on = True
                        to_return = False
                        ok_to_quit = True
                        print('Thank you for trying the movie recommendor system.')
                        print('Your results have not been saved.')
                        break
                    else:
                        # user did not enter a valid input to save or quit
                        print('That is not a valid input. Please Try again.')
                else:
                    # quitting the program without saving anything if dataframe is empty
                    print('Program is now closing.')
                    ok_to_move_on = True
                    rating = 10
                    to_return = False
                    ok_to_quit = True
                    break
            # letting user skip a movie review if they havent seen it
            elif rating == 'n':
                print('\n')
                break

            else:
                try:
                    # trying to convert rating input to float
                    rating = float(rating)
                    if rating not in range(1,6):
                        # letting user know that they need to use 1-5
                        print('That is not a valid rating, please enter a value 1-5:')
                        continue
                    else:
                        # saving user rating to temp df 
                        rating_dict['movieId'] = movie_review['movieId'].values[0]
                        rating_dict['rating'] = rating
                        rating_dict['title'] = movie_review['title'].values[0]
                        rating_dict['genres'] = movie_review['genres'].values[0]
                        current_session_df = current_session_df.append(rating_dict, ignore_index=True)
                        rating = 0
                        ok_to_move_on = False
                        print('Saving rating to temporary memory...\n\n')
                        break
                except:
                    # user tried to enter something other than 1-5, 'n', or 'quit'
                    print('That is not a valid rating, please enter a value 1-5:')
    
    # returns dataframe if they made changes to it, otherwise closes with no changes
    if to_return:
#         new_user_df = main_df
        return temp_df

In [58]:
def check_for_recs(user_id, username, password, main_df= new_user_df):
    if len(main_df[main_df['userId'] == user_id]) < 5:
        temp_df = movie_recs(user_id, username, password, main_df)
    else:
        make_more_recs = input('Would you like to make more recommendations? (yes/no): ')
        if make_more_recs == 'yes':
            temp_df = movie_recs(user_id, username, password, main_df)
        elif make_more_recs == 'no':
            pass
        else:
            print('You did not make a valid selection.')
           
       
    return temp_df

In [59]:
# old start program, replaced later on in notebook.
# will delate this once confirmed no bugs in new one

def start_program():
    user_id, username, password = new_user()
    
    if user_id is not np.NaN:    
        temp_df = check_for_recs(user_id, username, password)
    
        return temp_df
    else:
        print('Program was terminated.')

In [60]:
new_user_df = start_program()

(Type 'quit' at any time to exit.) 

Are you a new user? (yes/no): yes
Please enter a unique username: matt
Please enter a password: matt

Welcome! Your new account is created...

You need at least 5 reviews before we can make recommendations.

(Type 'quit' to stop rating movies.) 

Please rate the movie 1-5. If you have not seen the movie, type "n" to skip: 

['Maverick (1994)']
Rating: 
1
Saving rating to temporary memory...


You need at least 5 reviews before we can make recommendations.

(Type 'quit' to stop rating movies.) 

Please rate the movie 1-5. If you have not seen the movie, type "n" to skip: 

['Star Trek II: The Wrath of Khan (1982)']
Rating: 
1
Saving rating to temporary memory...


You need at least 5 reviews before we can make recommendations.

(Type 'quit' to stop rating movies.) 

Please rate the movie 1-5. If you have not seen the movie, type "n" to skip: 

['Die Hard 2 (1990)']
Rating: 
1
Saving rating to temporary memory...


You need at least 5 reviews before w

In [61]:
new_user_df

,userId,movieId,rating,title,genres,split_genres,username,password
0,1.0,1.0,4.0,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
1,5.0,1.0,4.0,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
2,7.0,1.0,4.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
3,15.0,1.0,2.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
4,17.0,1.0,4.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
...,...,...,...,...,...,...,...,...
100838,612.0,1374.0,1.0,Star Trek II: The Wrath of Khan (1982),action|adventure|sci-fi|thriller,NaN,matt,matt
100839,612.0,1370.0,1.0,Die Hard 2 (1990),action|adventure|thriller,NaN,matt,matt
100840,612.0,2006.0,1.0,"Mask of Zorro, The (1998)",action|comedy|romance,NaN,matt,matt
100841,612.0,40815.0,1.0,Harry Potter and the Goblet of Fire (2005),adventure|fantasy|thriller|imax,NaN,matt,matt


In [62]:
def make_rec(user_id, username, password, main_df = new_user_df):
    # list of all available genres
    genre_list = ['romance','fantasy','mystery','children','war','comedy',
                  'documentary','film-noir','thriller','action','sci-fi',
                  'drama','horror','crime','imax','animation','adventure',
                  'musical','western']
    # creating a while loop to keep the function opn
    keep_open = True
    print("(Enter 'quit' at any time to end the program.)")
    while keep_open == True:
        # seeing if user wants to review their recs
        rec = input('Would you like to retrieve your recommendations? (yes/no): ')
        if rec == 'yes':
            # Does the user want to limit results by genre?
            genre = input('Would you like recommendations from a specific genre? (yes/no): ')
            print('Compiling your recommendations...')
            # letting the user stop the program if they want to quit
            if genre == 'quit':
                print('Thank you for using the recommendation system.')
                print('Closing the program...')
                keep_open = False
                break      
            # If the user wants to limit results by genre
            elif genre == 'yes':
                print('Please select from the following list:')
                print('\t- romance','\n\t- fantasy','\n\t- mystery',
                      '\n\t- children','\n\t- war','\n\t- comedy',
                      '\n\t- documentary','\n\t- film-noir','\n\t- thriller',
                      '\n\t- action','\n\t- sci-fi','\n\t- drama','\n\t- horror',
                      '\n\t- crime','\n\t- imax','\n\t- animation','\n\t- adventure',
                      '\n\t- musical','\n\t- western')
                # Letting the user input specific genre
                genre_choice = input('Selection: ')
                # letting the user quit if they want to stop the program
                if genre_choice == 'quit':
                    print('Thank you for using the recommendation system.')
                    print('Closing the program...')
                    keep_open = False
                    break
                # Running the model with specified genre    
                elif genre_choice in genre_list:
                    print('Searching through specified genre...')
                    number_of_predictions = input('How many recommendations would you like? (1-50): ')
                    if number_of_predictions == 'quit':
                        print('Thank you for using the recommendation system.')
                        print('Closing the program...')
                        keep_open = False
                        break
                    else:
                        try:
                            number_of_predictions = int(number_of_predictions)
                            if number_of_predictions in list(range(1, 51)):
                                print('Processing request...')
                                rec_df = final_rec(user_id, genre=genre_choice, number_of_predictions=number_of_predictions)
                            else:
                                print('You did not select a valid response.')
                        except:
                            print('error: You did not select a valid response.')
                #telling the user that they didnt input valid response    
                else:
                    print('You did not select a valid response.')
            # running model with all genres
            elif genre == 'no':
                number_of_predictions = input('How many recommendations would you like? (1-50): ')
                if number_of_predictions == 'quit':
                    print('Thank you for using the recommendation system.')
                    print('Closing the program...')
                    keep_open = False
                    break
                else:
                    try:
                        number_of_predictions = int(number_of_predictions)
                        if number_of_predictions in list(range(1, 51)):
                            print('Processing request...')
                            rec_df = final_rec(user_id, number_of_predictions=number_of_predictions)
                        else:
                            print('You did not select a valid response.')
                    except:
                        print('error: You did not select a valid response.')
            else:
                # user did not enter a valid response
                print('You did not select a valid response.')
            
            
        # user does not want to receive recommendation now    
        elif rec == 'no':
            close_rec = input('Would you like to close the program? (yes/no): ')
            # closing recommendation
            if close_rec =='yes':
                print('Thank you for using the recommendation system.')
                print('Closing the program...')
                keep_open = False
                break
            # resetting so the user can get a rec if they want
            elif close_rec == 'no':
                continue
            #user did not input valid response
            else:
                print('You did not select a valid response.')
        # letting user quit program
        elif rec == 'quit':
            print('Thank you for using the recommendation system.')
            print('Closing the program...')
            keep_open = False
            break
        # user did not submit valid response
        else:
            print('You did not select a valid response.')
    
    return main_df

In [63]:
def final_rec(user_id, genre = None, number_of_predictions= 10, main_df = new_user_df, movies_df = movies_df, model = baseline):
    # creating a dataframe with a specified user
    genre_list = ['romance','fantasy','mystery','children','war','comedy',
                  'documentary','film-noir','thriller','action','sci-fi',
                  'drama','horror','crime','imax','animation','adventure',
                  'musical','western']
    if genre == None:
        user_ratings = main_df[main_df['userId'] == user_id][['userId', 'movieId', 'rating']]
        # creating a dataframe with movies not yet reviewed
        user_predict = list(movies_df[~movies_df['movieId'].isin(user_ratings['movieId'].values)].movieId.values)
        # creating predictions and storing them in list
        predictions = []
        for movie in user_predict:
            predictions.append((movies_df['title'].loc[movies_df['movieId'] == movie].values[0], \
                                model.predict(user_id, movie)[3]))
        
        predicted_df = pd.DataFrame(predictions, columns=['title', 'predicted rating']).sort_values('predicted rating', ascending=False).head(number_of_predictions)
        print(predicted_df)
    
    elif genre in genre_list:
        user_ratings = main_df[main_df['userId'] == user_id][['userId', 'movieId', 'rating']]
        # creating a dataframe with movies not yet reviewed
        user_predict = list(movies_df[~movies_df['movieId'].isin(user_ratings['movieId'].values)].movieId.values)
        # creating predictions and storing them in list
        predictions = []
        for movie in user_predict:
            predictions.append((movies_df['title'].loc[movies_df['movieId'] == movie].values[0], \
                                model.predict(user_id, movie)[3]))
        
        predicted_df = pd.DataFrame(predictions, columns=['title', 'predicted rating']).sort_values('predicted rating', ascending=False)
        final_prediction = predicted_df.merge(movies_df).drop('movieId', axis=1)
        predicted_df = final_prediction[final_prediction['genres'].str.contains(genre)].drop(['genres', 'split_genres'], axis=1).head(number_of_predictions)
        print(predicted_df)
    else:
        print('You did not select a valid response.')
      
#     new_user_df = main_df
    return main_df

In [64]:
# number_of_predictions = 10

In [65]:
# final_rec(612, genre='comedy', number_of_predictions=number_of_predictions)

In [66]:
# make_rec(612, 'matt', 'matt')

In [64]:
def start_program():
    user_id, username, password = new_user()
    
    if user_id is not np.NaN:    
        temp_df = check_for_recs(user_id, username, password)
        
        temp_df= make_rec(user_id, username, password, main_df=temp_df)
        
        
        return temp_df
    else:
        print('Program was terminated.')

In [71]:
new_user_df = start_program()

(Type 'quit' at any time to exit.) 

Are you a new user? (yes/no): yes
Please enter a unique username: jon
Please enter a password: jon

Welcome! Your new account is created...

You need at least 5 reviews before we can make recommendations.

(Type 'quit' to stop rating movies.) 

Please rate the movie 1-5. If you have not seen the movie, type "n" to skip: 

['American President, The (1995)']
Rating: 
3
Saving rating to temporary memory...


You need at least 5 reviews before we can make recommendations.

(Type 'quit' to stop rating movies.) 

Please rate the movie 1-5. If you have not seen the movie, type "n" to skip: 

['Shutter Island (2010)']
Rating: 
3
Saving rating to temporary memory...


You need at least 5 reviews before we can make recommendations.

(Type 'quit' to stop rating movies.) 

Please rate the movie 1-5. If you have not seen the movie, type "n" to skip: 

['Godfather, The (1972)']
Rating: 
3
Saving rating to temporary memory...


You need at least 5 reviews before w

In [72]:
new_user_df

,userId,movieId,rating,title,genres,split_genres,username,password
0,1.0,1.0,4.0,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
1,5.0,1.0,4.0,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
2,7.0,1.0,4.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
3,15.0,1.0,2.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
4,17.0,1.0,4.5,Toy Story (1995),adventure|animation|children|comedy|fantasy,"[adventure, animation, children, comedy, fantasy]",NaN,NaN
...,...,...,...,...,...,...,...,...
100839,613.0,858.0,3.0,"Godfather, The (1972)",crime|drama,NaN,jon,jon
100840,613.0,4816.0,3.0,Zoolander (2001),comedy,NaN,jon,jon
100841,613.0,4643.0,3.0,Planet of the Apes (2001),action|adventure|drama|sci-fi,NaN,jon,jon
100842,613.0,317.0,3.0,"Santa Clause, The (1994)",comedy|drama|fantasy,NaN,jon,jon


## Go back tomorrow and check output for all functions to see why it isnt saving to the dataframe

In [70]:
# working baseline

def movie_recs(user_id, username, password, main_df = new_user_df):
    # Setting the base rating at zero
    rating = 0
    # creating a new DF to hold the temp ratings
    current_session_df = pd.DataFrame()
    # whether or not its ok to quit the recommend prompt loop
    ok_to_quit = False
    # automatically puts user in the loop if they have less than 5 ratings
    if len(new_user_df[new_user_df['userId'] == user_id]) < 5:
        # the loop to make recommendations
        while ok_to_quit is False:
            print("You need at least 5 reviews before we can make recommendations.")
            # setting a loop to keep sampling if the sample is already in the main df or in the temp df
            ok_to_move_on = False
            while ok_to_move_on is False:
                # pulling sample from most rated moves
                movie_review = top_100.sample()
                movie_check = movie_review['movieId'].values
                # checking length of df to see if its been filled yet
                # ensuring movie has not already been rated
                if len(current_session_df) != 0:
                    if movie_check not in new_user_df[new_user_df['userId'] == 
                                                      user_id]['movieId'].values and\
                    movie_check not in current_session_df['movieId'].values:
                        ok_to_move_on = True
                    # resample if already rated
                    else:
                        continue
                else:
                    # ensuring movie has not already been rated
                    if movie_check not in new_user_df[new_user_df['userId'] == 
                                                      user_id]['movieId'].values:
                        ok_to_move_on = True
                    else:
                        # resample if already rated
                        continue
            # creating a loop to ensure that user provides a valid rating
            while rating not in range(1,6):
                # creating a dictionary to store rating info that will be appended to df
                rating_dict = {'userId':user_id, 
                           'movieId': np.nan, 
                           'rating': np.nan, 
                           'title': np.nan, 
                           'genres': np.nan, 
                           'username': username, 
                           'password': password}
                print(f"\n(Type 'quit' to stop rating movies.) \n")
                # telling user how to rate
                print(f'Please rate the movie 1-5. If you have not seen the movie, type "n" to skip: \n')
                # giving user movie info
                print(f"{movie_review['title'].values}")
                rating = input('Rating: \n')
                # letting the user quit if they dont want to rate
                if rating == 'quit':
                    print('\n\nThank you for ranking movies.')
                    # checking to see if the user made any ratings during this session
                    if len(current_session_df) != 0:
                        print('Here are the results of your ranking session:')
                        # showing the user the ratings they made
                        print(current_session_df[['title', 'rating']])
                        # checking if user wants to save ratings
                        submit_ratings = input('\nWould you like to save these ratings to your profile? (yes/quit): ')
                        if submit_ratings == 'yes':
                            # saving ratings to main database and closing program
                            main_df = main_df.append(current_session_df, ignore_index=True)
                            ok_to_quit = True
                            ok_to_move_on = True
                            to_return = True
                            break
                        elif submit_ratings == 'quit':
                            # not saving ratings to main database and closing program
                            rating = 10
                            ok_to_move_on = True
                            to_return = False
                            ok_to_quit = True
                            print('Thank you for trying the movie recommendor system.')
                            print('Your results have not been saved.')
                            break
                        else:
                            # user did not enter a valid input to save or quit
                            print('That is not a valid input. Please Try again.')
                    else:
                        # quitting the program without saving anything if dataframe is empty
                        print('Program is now closing.')
                        ok_to_move_on = True
                        rating = 10
                        to_return = False
                        ok_to_quit = True
                        break
                # letting user skip a movie review if they havent seen it
                elif rating == 'n':
                    print('\n')
                    break

                else:
                    try:
                        # trying to convert rating input to float
                        rating = float(rating)
                        if rating not in range(1,6):
                            # letting user know that they need to use 1-5
                            print('That is not a valid rating, please enter a value 1-5:')
                            continue
                        else:
                            # saving user rating to temp df 
                            rating_dict['movieId'] = movie_review['movieId'].values[0]
                            rating_dict['rating'] = rating
                            rating_dict['title'] = movie_review['title'].values[0]
                            rating_dict['genres'] = movie_review['genres'].values[0]
                            current_session_df = current_session_df.append(rating_dict, ignore_index=True)
                            rating = 0
                            ok_to_move_on = False
                            print('Saving rating to temporary memory...\n\n')
                            break
                    except:
                        # user tried to enter something other than 1-5, 'n', or 'quit'
                        print('That is not a valid rating, please enter a value 1-5:')
    
    # returns dataframe if they made changes to it, otherwise closes with no changes
    if to_return:    
        return main_df

## making predictions with surprise

In [71]:
user_to_rate = 2

In [72]:
# creating a dataframe with a specified user
user_ratings = new_user_df[new_user_df['userId'] == user_to_rate][['userId', 'movieId', 'rating']]
user_ratings.head()

,userId,movieId,rating
2267,2.0,333,4.0
15657,2.0,3578,4.0
16296,2.0,318,3.0
16613,2.0,1704,4.5
16754,2.0,6874,4.0


In [73]:
user_ratings['movieId'].values

array([   333,   3578,    318,   1704,   6874,   8798,  46970,  48516,
        58559,  60756,  68157,  71535,  74458,  77455,  79132,  80489,
        80906,  86345,  89774,  91529,  91658,  99114, 106782, 109487,
       112552, 114060, 115713, 122882, 131724], dtype=int64)

In [74]:
# creating a dataframe with movies not yet reviewed
user_predict = list(movies_df[~movies_df['movieId'].isin(user_ratings['movieId'].values)].movieId.values)
# user_predict

In [75]:
predictions = []
for movie in user_predict:
    predictions.append((movies_df['title'].loc[movies_df['movieId'] == movie].values[0], \
                        baseline.predict(user_to_rate, movie)[3]))
    

In [76]:
predicted_df = pd.DataFrame(predictions, columns=['title', 'prediction'])

In [77]:
final_prediction = predicted_df.merge(movies_df).drop('movieId', axis=1)

In [78]:
test = final_prediction.sort_values('prediction', ascending=False)

In [79]:
print(test[test['genres'].str.contains(('romance').title())])

Empty DataFrame
Columns: [title, prediction, genres, split_genres]
Index: []


In [80]:
test

,title,prediction,genres,split_genres
899,Star Wars: Episode V - The Empire Strikes Back...,4.411833,action|adventure|sci-fi,"[action, adventure, sci-fi]"
2229,Fight Club (1999),4.386436,action|crime|drama|thriller,"[action, crime, drama, thriller]"
600,Dr. Strangelove or: How I Learned to Stop Worr...,4.383926,comedy|war,"[comedy, war]"
907,Lawrence of Arabia (1962),4.363100,adventure|drama|war,"[adventure, drama, war]"
660,"Godfather, The (1972)",4.349665,crime|drama,"[crime, drama]"
...,...,...,...,...
2029,Wild Wild West (1999),2.700796,action|comedy|sci-fi|western,"[action, comedy, sci-fi, western]"
5274,Catwoman (2004),2.699413,action|crime|fantasy,"[action, crime, fantasy]"
1373,Godzilla (1998),2.661489,action|sci-fi|thriller,"[action, sci-fi, thriller]"
1312,Spice World (1997),2.650159,comedy,[comedy]


In [81]:
predicted_df.sort_values('prediction', ascending=False).head(10)

,title,prediction
896,Star Wars: Episode V - The Empire Strikes Back...,4.411833
2223,Fight Club (1999),4.386436
600,Dr. Strangelove or: How I Learned to Stop Worr...,4.383926
904,Lawrence of Arabia (1962),4.363100
657,"Godfather, The (1972)",4.349665
692,Casablanca (1942),4.341353
897,"Princess Bride, The (1987)",4.324890
46,"Usual Suspects, The (1995)",4.321951
907,Apocalypse Now (1979),4.315185
224,Star Wars: Episode IV - A New Hope (1977),4.313092


In [82]:
predictions

[('Toy Story (1995)', 4.013543396286959),
 ('Jumanji (1995)', 3.647139404330795),
 ('Grumpier Old Men (1995)', 3.4558718534655095),
 ('Waiting to Exhale (1995)', 3.2635810556818687),
 ('Father of the Bride Part II (1995)', 3.294237100026675),
 ('Heat (1995)', 4.022059772354651),
 ('Sabrina (1995)', 3.299375022715588),
 ('Tom and Huck (1995)', 3.4883789531691876),
 ('Sudden Death (1995)', 3.390387011401443),
 ('GoldenEye (1995)', 3.608037212980662),
 ('American President, The (1995)', 3.8109337117224835),
 ('Dracula: Dead and Loving It (1995)', 2.9912268941425424),
 ('Balto (1995)', 3.5307718077826493),
 ('Nixon (1995)', 3.7045657795492493),
 ('Cutthroat Island (1995)', 3.3893698114101047),
 ('Casino (1995)', 3.9792277892036885),
 ('Sense and Sensibility (1995)', 3.9994613818479885),
 ('Four Rooms (1995)', 3.775151527206968),
 ('Ace Ventura: When Nature Calls (1995)', 2.9582319004933293),
 ('Money Train (1995)', 3.1800508036538364),
 ('Get Shorty (1995)', 3.6990030188928578),
 ('Copycat

In [83]:
movies_df['split_genres'] = movies_df['genres'].iloc[0:].apply(lambda x: x.lower().split('|'))

movies = []
for i in movies_df['split_genres']:
    for x in i:
        movies.append(x)

In [84]:
list(set(movies))

['thriller',
 'romance',
 'crime',
 'fantasy',
 'animation',
 'western',
 'children',
 'film-noir',
 'musical',
 '(no genres listed)',
 'horror',
 'action',
 'sci-fi',
 'war',
 'drama',
 'adventure',
 'mystery',
 'comedy',
 'documentary',
 'imax']

In [85]:
tags_df['tag_clean'] = tags_df['tag'].iloc[0:].apply(lambda x: x.lower().replace(' ', ''))
tags_df['tag_clean'].head()

0             funny
1    highlyquotable
2       willferrell
3       boxingstory
4               mma
Name: tag_clean, dtype: object

## Final Model

## Results/Conclusions

### Further Exploration/Questions